# Data Preparation

This notebook prepares data for the thesis.

We use only these files for now:
- `brogmkenset.gpkg`
- `bro_groundwater_use.gpkg`
- `era5_land_monthly.nc`


## Imports and paths

In [11]:
from pathlib import Path

import numpy as np
import pandas as pd
import geopandas as gpd
import xarray as xr
import fiona

ROOT = Path.cwd()

MONITOR_GPKG = ROOT / "Data" / "raw" / "brogmkenset.gpkg"
USE_GPKG = ROOT / "Data" / "raw" / "bro_groundwater_use.gpkg"
ERA5_NC = ROOT / "Data" / "raw" / "era5_land_monthly.nc"

for p in [MONITOR_GPKG, USE_GPKG, ERA5_NC]:
    if not p.exists():
        raise FileNotFoundError(f"Missing file: {p}")

print("Using files:")
print("-", MONITOR_GPKG)
print("-", USE_GPKG)
print("-", ERA5_NC)


def read_table_gpkg(gpkg_path: Path, layer: str) -> pd.DataFrame:
    # Read a non-spatial GPKG table as pandas DataFrame (no SQL).
    # Fiona may omit primary key columns in properties, so we recover them from feat.id.
    rows = []
    pk_col = f"{layer}_pk"

    with fiona.open(gpkg_path, layer=layer) as src:
        for feat in src:
            rec = dict(feat["properties"])
            fid = feat.get("id")
            if fid is not None:
                try:
                    rec["_fid"] = int(fid)
                except Exception:
                    rec["_fid"] = fid
            rows.append(rec)

    df = pd.DataFrame(rows)
    if pk_col not in df.columns and "_fid" in df.columns:
        df[pk_col] = df["_fid"]
    return df


def require_columns(df: pd.DataFrame, required: list[str], name: str) -> None:
    missing = [c for c in required if c not in df.columns]
    if missing:
        raise KeyError(f"{name} is missing columns: {missing}")


Using files:
- /Users/izhan/Desktop/groundwater/stephanie_thesis_groundwater/Data/raw/brogmkenset.gpkg
- /Users/izhan/Desktop/groundwater/stephanie_thesis_groundwater/Data/raw/bro_groundwater_use.gpkg
- /Users/izhan/Desktop/groundwater/stephanie_thesis_groundwater/Data/raw/era5_land_monthly.nc


## Select monitoring sites in ZH

Rule:
- inside rough ZH bbox
- enough records (`max_observations >= 120`)
- long time coverage (`first_date <= 2010-01-01`, `last_date >= 2025-01-01`)

In [12]:
# Load monitoring layers
# Note: some environments do not expose FID columns (like *_pk) as normal columns.
tubes = gpd.read_file(MONITOR_GPKG, layer="gm_gmw_monitoringtube")
series = gpd.read_file(MONITOR_GPKG, layer="gm_gld")

if "gm_gmw_monitoringtube_pk" not in tubes.columns:
    tubes = tubes.reset_index().rename(columns={"index": "gm_gmw_monitoringtube_pk"})

require_columns(tubes, ["gm_gmw_monitoringtube_pk", "gm_gmw_fk", "geometry"], "gm_gmw_monitoringtube")
require_columns(series, ["gm_gmw_monitoringtube_fk", "research_first_date", "research_last_date", "number_of_observations"], "gm_gld")

# Parse dates
series["research_first_date"] = pd.to_datetime(series["research_first_date"], errors="coerce")
series["research_last_date"] = pd.to_datetime(series["research_last_date"], errors="coerce")
series["number_of_observations"] = pd.to_numeric(series["number_of_observations"], errors="coerce")

# One metadata row per tube
# Use size() so we do not depend on gm_gld_pk being present.
series_by_tube = (
    series.groupby("gm_gmw_monitoringtube_fk", as_index=False)
    .agg(
        n_series=("gm_gmw_monitoringtube_fk", "size"),
        max_observations=("number_of_observations", "max"),
        first_date=("research_first_date", "min"),
        last_date=("research_last_date", "max"),
    )
)

sites = tubes[["gm_gmw_monitoringtube_pk", "gm_gmw_fk", "geometry"]].copy()
sites = sites.merge(
    series_by_tube,
    left_on="gm_gmw_monitoringtube_pk",
    right_on="gm_gmw_monitoringtube_fk",
    how="inner",
)
sites = sites.rename(columns={
    "gm_gmw_monitoringtube_pk": "site_id",
    "gm_gmw_fk": "well_id",
})

# Rough ZH bbox (lon/lat)
minx, maxx = 3.6, 5.1
miny, maxy = 51.7, 52.3
sites_zh = sites.cx[minx:maxx, miny:maxy].copy()

# Keep strong candidate sites
sites_zh = sites_zh[
    (sites_zh["max_observations"] >= 120)
    & (sites_zh["first_date"] <= pd.Timestamp("2010-01-01"))
    & (sites_zh["last_date"] >= pd.Timestamp("2025-01-01"))
].copy()

if sites_zh.empty:
    raise ValueError("No sites left after filtering. Relax bbox/date/observation rules.")

sites_zh["lon"] = sites_zh.geometry.x
sites_zh["lat"] = sites_zh.geometry.y

print("Total tubes with series:", sites["site_id"].nunique())
print("ZH candidate tubes:", sites_zh["site_id"].nunique())
print("ZH candidate wells:", sites_zh["well_id"].nunique())
print(sites_zh[["site_id", "well_id", "max_observations", "first_date", "last_date"]].head())


Total tubes with series: 94182
ZH candidate tubes: 649
ZH candidate wells: 351
      site_id  well_id  max_observations first_date  last_date
5652     9708     8538               158 1997-09-15 2025-10-01
5653     9709     8538               354 1997-09-15 2025-10-01
5664     9722     8546               196 1993-03-18 2025-09-25
5665     9723     8546               196 1993-03-18 2025-09-25
5670     9728     8549               442 1987-01-14 2025-09-25


## Prepare monthly ERA5 features for each site

Features:
- `tp_mm` (precipitation, mm/month)
- `pet_mm` (potential evaporation, mm/month, positive direction)
- `t2m_c` (2m temperature, Celsius)
- `spi3_z`, `spi6_z`, `spi12_z` (simple SPI-like z-score from rolling precipitation sums)

In [13]:
# Time window used for model inputs
MONTHS = pd.date_range("2010-01-01", "2025-12-01", freq="MS")

# Load ERA5 monthly data
era5 = xr.open_dataset(ERA5_NC)
era5 = era5.sel(valid_time=slice(MONTHS.min(), MONTHS.max()))

site_ids = sites_zh["site_id"].astype(int).to_numpy()
lon_da = xr.DataArray(
    sites_zh.set_index("site_id").loc[site_ids, "lon"].to_numpy(),
    dims=["site_id"],
    coords={"site_id": site_ids},
)
lat_da = xr.DataArray(
    sites_zh.set_index("site_id").loc[site_ids, "lat"].to_numpy(),
    dims=["site_id"],
    coords={"site_id": site_ids},
)

# Nearest ERA5 grid cell for each site
meteo = xr.Dataset(
    {
        "tp_mm": era5["tp"].sel(longitude=lon_da, latitude=lat_da, method="nearest") * 1000.0,
        "pet_mm": -era5["pev"].sel(longitude=lon_da, latitude=lat_da, method="nearest") * 1000.0,
        "t2m_c": era5["t2m"].sel(longitude=lon_da, latitude=lat_da, method="nearest") - 273.15,
    }
)

# Simple SPI-like z-score from rolling precipitation sums
for w in [3, 6, 12]:
    p_roll = meteo["tp_mm"].rolling(valid_time=w, min_periods=w).sum()
    meteo[f"spi{w}_z"] = (p_roll - p_roll.mean(dim="valid_time")) / (p_roll.std(dim="valid_time") + 1e-6)

meteo_df = meteo.to_dataframe().reset_index()
meteo_df = meteo_df.rename(columns={"valid_time": "month"})
meteo_df["month"] = pd.to_datetime(meteo_df["month"]).dt.to_period("M").dt.to_timestamp()

print("Meteo table shape:", meteo_df.shape)
print(meteo_df.head())

Meteo table shape: (124608, 12)
       month  site_id  number  latitude  longitude expver     tp_mm    pet_mm  \
0 2010-01-01     9708       0      52.2        5.1   0001  1.420961  0.474006   
1 2010-01-01     9709       0      52.2        5.1   0001  1.420961  0.474006   
2 2010-01-01     9722       0      52.3        4.8   0001  1.577557  0.599541   
3 2010-01-01     9723       0      52.3        4.8   0001  1.577557  0.599541   
4 2010-01-01     9728       0      52.3        4.8   0001  1.577557  0.599541   

      t2m_c  spi3_z  spi6_z  spi12_z  
0 -0.712128     NaN     NaN      NaN  
1 -0.712128     NaN     NaN      NaN  
2 -0.161346     NaN     NaN      NaN  
3 -0.161346     NaN     NaN      NaN  
4 -0.161346     NaN     NaN      NaN  


/Users/izhan/opt/anaconda3/envs/thsis/lib/python3.11/site-packages/numpy/lib/_nanfunctions_impl.py:2015: RuntimeWarning: Degrees of freedom <= 0 for slice.
  var = nanvar(a, axis=axis, dtype=dtype, out=out, ddof=ddof,
/Users/izhan/opt/anaconda3/envs/thsis/lib/python3.11/site-packages/numpy/lib/_nanfunctions_impl.py:2015: RuntimeWarning: Degrees of freedom <= 0 for slice.
  var = nanvar(a, axis=axis, dtype=dtype, out=out, ddof=ddof,
/Users/izhan/opt/anaconda3/envs/thsis/lib/python3.11/site-packages/numpy/lib/_nanfunctions_impl.py:2015: RuntimeWarning: Degrees of freedom <= 0 for slice.
  var = nanvar(a, axis=axis, dtype=dtype, out=out, ddof=ddof,


## Build monthly pumping proxy (buffer-based)

Method:
1. Keep only `onttrekken` (abstraction) quantities.
2. Convert quantity to annual volume (`annual_m3`).
3. Link quantity to facility and validity period.
4. For each site, sum active facilities in a 5 km buffer each month.

This gives `pumping_m3_month`.

In [14]:
# Load facility geometry and tabular layers (no SQL)
facilities = gpd.read_file(USE_GPKG, layer="groundwater_usage_facility")
if "groundwater_usage_facility_pk" not in facilities.columns:
    facilities = facilities.reset_index().rename(columns={"index": "groundwater_usage_facility_pk"})
require_columns(facilities, ["groundwater_usage_facility_pk", "geometry"], "groundwater_usage_facility")
facilities = facilities[["groundwater_usage_facility_pk", "geometry"]].copy()

licence = read_table_gpkg(USE_GPKG, "licence_groundwater_usage")
quantity = read_table_gpkg(USE_GPKG, "licensed_quantity")
validity = read_table_gpkg(USE_GPKG, "validity_period")

require_columns(licence, ["licence_groundwater_usage_pk", "groundwater_usage_facility_fk"], "licence_groundwater_usage")
require_columns(
    quantity,
    [
        "licence_groundwater_usage_fk",
        "licensed_in_out",
        "maximum_per_hour",
        "maximum_per_day",
        "maximum_per_month",
        "maximum_per_year",
    ],
    "licensed_quantity",
)
require_columns(validity, ["groundwater_usage_facility_fk", "start_validity", "end_validity"], "validity_period")

licence = licence[["licence_groundwater_usage_pk", "groundwater_usage_facility_fk"]].copy()
quantity = quantity[[
    "licence_groundwater_usage_fk",
    "licensed_in_out",
    "maximum_per_hour",
    "maximum_per_day",
    "maximum_per_month",
    "maximum_per_year",
]].copy()
validity = validity[["groundwater_usage_facility_fk", "start_validity", "end_validity"]].copy()

# Keep only abstraction rows
quantity["licensed_in_out"] = quantity["licensed_in_out"].astype(str).str.lower().str.strip()
quantity = quantity[quantity["licensed_in_out"] == "onttrekken"].copy()

# Numeric conversion
for c in ["maximum_per_hour", "maximum_per_day", "maximum_per_month", "maximum_per_year"]:
    quantity[c] = pd.to_numeric(quantity[c], errors="coerce")

# Build annual volume (fallback if maximum_per_year is missing)
quantity["annual_m3"] = quantity["maximum_per_year"]
quantity["annual_m3"] = quantity["annual_m3"].fillna(quantity["maximum_per_month"] * 12)
quantity["annual_m3"] = quantity["annual_m3"].fillna(quantity["maximum_per_day"] * 365)
quantity["annual_m3"] = quantity["annual_m3"].fillna(quantity["maximum_per_hour"] * 24 * 365)
quantity["annual_m3"] = quantity["annual_m3"].fillna(0.0)
quantity["annual_m3"] = quantity["annual_m3"].clip(lower=0)

# Link licence -> facility
licence["licence_groundwater_usage_pk"] = pd.to_numeric(licence["licence_groundwater_usage_pk"], errors="coerce")
licence["groundwater_usage_facility_fk"] = pd.to_numeric(licence["groundwater_usage_facility_fk"], errors="coerce")
quantity["licence_groundwater_usage_fk"] = pd.to_numeric(quantity["licence_groundwater_usage_fk"], errors="coerce")

q_licence = quantity.groupby("licence_groundwater_usage_fk", as_index=False)["annual_m3"].sum()
q_facility = (
    licence.merge(q_licence, left_on="licence_groundwater_usage_pk", right_on="licence_groundwater_usage_fk", how="left")
    .groupby("groundwater_usage_facility_fk", as_index=False)["annual_m3"].sum()
)
q_facility["annual_m3"] = q_facility["annual_m3"].fillna(0.0)

# Validity cleaning and aggregation
validity["groundwater_usage_facility_fk"] = pd.to_numeric(validity["groundwater_usage_facility_fk"], errors="coerce")
validity["start_validity"] = pd.to_datetime(validity["start_validity"], errors="coerce")
validity["end_validity"] = pd.to_datetime(validity["end_validity"], errors="coerce")

validity.loc[validity["start_validity"].dt.year < 1900, "start_validity"] = pd.Timestamp("1900-01-01")
validity.loc[validity["end_validity"].dt.year > 2100, "end_validity"] = pd.Timestamp("2100-12-31")

validity["start_validity"] = validity["start_validity"].fillna(pd.Timestamp("1900-01-01"))
validity["end_validity"] = validity["end_validity"].fillna(pd.Timestamp("2100-12-31"))

validity_facility = validity.groupby("groundwater_usage_facility_fk", as_index=False).agg(
    valid_start=("start_validity", "min"),
    valid_end=("end_validity", "max"),
)

# Merge facility geometry + quantity + validity
facilities["groundwater_usage_facility_pk"] = pd.to_numeric(facilities["groundwater_usage_facility_pk"], errors="coerce")
facilities = facilities.merge(
    q_facility,
    left_on="groundwater_usage_facility_pk",
    right_on="groundwater_usage_facility_fk",
    how="left",
)
facilities = facilities.merge(
    validity_facility,
    left_on="groundwater_usage_facility_pk",
    right_on="groundwater_usage_facility_fk",
    how="left",
)

facilities["annual_m3"] = facilities["annual_m3"].fillna(0.0)
facilities["valid_start"] = facilities["valid_start"].fillna(pd.Timestamp("1900-01-01"))
facilities["valid_end"] = facilities["valid_end"].fillna(pd.Timestamp("2100-12-31"))

# MultiPoint -> Point rows
facilities = facilities.explode(index_parts=False, ignore_index=True)
facilities = facilities[facilities.geometry.notna()].copy()

# Keep ZH bbox and positive quantity only
facilities_zh = facilities.cx[minx:maxx, miny:maxy].copy()
facilities_zh = facilities_zh[facilities_zh["annual_m3"] > 0].copy()

print("Facilities in ZH with positive annual quantity:", len(facilities_zh))
print(facilities_zh[["groundwater_usage_facility_pk", "annual_m3", "valid_start", "valid_end"]].head())


Facilities in ZH with positive annual quantity: 5808
     groundwater_usage_facility_pk  annual_m3 valid_start  valid_end
25                              22    25000.0  2023-03-23 2100-12-31
50                              42  1071360.0  2023-03-14 2100-12-31
91                              61   311000.0  2023-02-22 2100-12-31
151                             93      567.0  2023-02-01 2023-03-06
153                             95    12800.0  2022-06-06 2100-12-31


In [15]:
# Spatial join: site buffer (5 km) and facility points
BUFFER_METERS = 5000

sites_m = sites_zh[["site_id", "geometry"]].to_crs(28992)
facilities_m = facilities_zh[[
    "groundwater_usage_facility_pk",
    "annual_m3",
    "valid_start",
    "valid_end",
    "geometry",
]].to_crs(28992)

site_buffers = sites_m.copy()
site_buffers["geometry"] = site_buffers.geometry.buffer(BUFFER_METERS)

pairs = gpd.sjoin(
    facilities_m,
    site_buffers[["site_id", "geometry"]],
    how="inner",
    predicate="within",
)

pairs = pairs[["groundwater_usage_facility_pk", "site_id", "annual_m3", "valid_start", "valid_end"]]
pairs = pairs.drop_duplicates(subset=["groundwater_usage_facility_pk", "site_id"]).copy()

pairs["valid_start"] = pd.to_datetime(pairs["valid_start"], errors="coerce").dt.to_period("M").dt.to_timestamp()
pairs["valid_end"] = pd.to_datetime(pairs["valid_end"], errors="coerce").dt.to_period("M").dt.to_timestamp()

pairs["valid_start"] = pairs["valid_start"].fillna(pd.Timestamp("1900-01-01"))
pairs["valid_end"] = pairs["valid_end"].fillna(pd.Timestamp("2100-12-01"))

# Expand pairs to month and keep active months only
months_df = pd.DataFrame({"month": MONTHS})
months_df["_k"] = 1
pairs["_k"] = 1
pair_month = pairs.merge(months_df, on="_k", how="inner").drop(columns=["_k"])

pair_month = pair_month[
    (pair_month["month"] >= pair_month["valid_start"])
    & (pair_month["month"] <= pair_month["valid_end"])
].copy()

pair_month["pumping_m3_month"] = pair_month["annual_m3"] / 12.0

pumping_df = (
    pair_month.groupby(["site_id", "month"], as_index=False)["pumping_m3_month"]
    .sum()
)

print("Site-facility pairs:", len(pairs))
print("Monthly pumping rows:", len(pumping_df))
print(pumping_df.head())

Site-facility pairs: 25488
Monthly pumping rows: 91183
   site_id      month  pumping_m3_month
0     9708 2010-01-01           68525.0
1     9708 2010-02-01           68525.0
2     9708 2010-03-01           68525.0
3     9708 2010-04-01           68525.0
4     9708 2010-05-01           68525.0


## Build final panel and save

Output:
- `Data/processed/zh_monthly_features.parquet`
- `Data/processed/zh_sites.parquet`

In [16]:
# Build site-month skeleton
panel = pd.MultiIndex.from_product(
    [sites_zh["site_id"].astype(int).unique(), MONTHS],
    names=["site_id", "month"],
).to_frame(index=False)

# Merge features
panel = panel.merge(meteo_df, on=["site_id", "month"], how="left")
panel = panel.merge(pumping_df, on=["site_id", "month"], how="left")
panel["pumping_m3_month"] = panel["pumping_m3_month"].fillna(0.0)

# Add static site metadata
site_meta = sites_zh[[
    "site_id", "well_id", "lon", "lat", "max_observations", "first_date", "last_date"
]].copy()
site_meta["site_id"] = site_meta["site_id"].astype(int)

panel = panel.merge(site_meta, on="site_id", how="left")

# Save
out_dir = ROOT / "Data" / "processed"
out_dir.mkdir(parents=True, exist_ok=True)

panel_path = out_dir / "zh_monthly_features.parquet"
sites_path = out_dir / "zh_sites.parquet"

saved_main = None
saved_meta = None

try:
    panel.to_parquet(panel_path, index=False)
    site_meta.to_parquet(sites_path, index=False)
    saved_main = panel_path
    saved_meta = sites_path
except Exception:
    panel_path = out_dir / "zh_monthly_features.pkl"
    sites_path = out_dir / "zh_sites.pkl"
    panel.to_pickle(panel_path)
    site_meta.to_pickle(sites_path)
    saved_main = panel_path
    saved_meta = sites_path

print("Saved:")
print("-", saved_main)
print("-", saved_meta)
print("Panel shape:", panel.shape)
print(panel.head())

Saved:
- /Users/izhan/Desktop/groundwater/stephanie_thesis_groundwater/Data/processed/zh_monthly_features.parquet
- /Users/izhan/Desktop/groundwater/stephanie_thesis_groundwater/Data/processed/zh_sites.parquet
Panel shape: (124608, 19)
   site_id      month  number  latitude  longitude expver     tp_mm    pet_mm  \
0     9708 2010-01-01       0      52.2        5.1   0001  1.420961  0.474006   
1     9708 2010-02-01       0      52.2        5.1   0001  3.182507  1.163788   
2     9708 2010-03-01       0      52.2        5.1   0001  2.071045  2.689257   
3     9708 2010-04-01       0      52.2        5.1   0001  1.068072  4.618317   
4     9708 2010-05-01       0      52.2        5.1   0001  2.236217  4.809119   

       t2m_c    spi3_z  spi6_z  spi12_z  pumping_m3_month  well_id       lon  \
0  -0.712128       NaN     NaN      NaN           68525.0     8538  5.067566   
1   1.336090       NaN     NaN      NaN           68525.0     8538  5.067566   
2   5.780853 -0.071914     NaN      N

## to do next

Build the target variable 
